In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import os
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score
from keras.layers import TFSMLayer

2024-12-05 16:36:10.070503: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733412970.084263   24936 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733412970.088428   24936 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 16:36:10.102203: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Model size

In [4]:
model_tflite_size = os.path.getsize('qat_model.tflite')/(1024**2)
model_original_size = sum(os.path.getsize(os.path.join(dirpath, filename)) for dirpath, _, filenames in os.walk('original_model') for filename in filenames)/(1024**2)

In [5]:
print(f"Original model size: {model_original_size:.2f} MB")
print(f"Quantized model size: {model_tflite_size:.2f} MB")

Original model size: 119.30 MB
Quantized model size: 9.91 MB


### Inference time

Time for inference to compare the QAT model vs. the original one has been executed on the same device with CPUs, note that the time on GAP9 may be different. The test was executed through the time_test.py script.

In [6]:
print('Original model mean time per inference: 30.58 ms')
print('Quantized model mean time per inference: 13.99 ms')

Original model mean time per inference: 30.58 ms
Quantized model mean time per inference: 13.99 ms


### Performance metrics

In [7]:
def dataset_creator(path_to_folder, max_by_experiment = 3):
    col, fire, steer = 0, 0, 0
    images, labels = [], []

    for folder in os.listdir(path_to_folder):

        if os.path.isdir(os.path.join(path_to_folder, folder)):

            for file in os.listdir(os.path.join(path_to_folder, folder)):

                if file.endswith('.txt'):
                    if 'labels' in file and col < max_by_experiment:
                        col += 1
                        for pic in sorted(os.listdir(os.path.join(path_to_folder, folder, 'images'))):
                            img = load_img(os.path.join(os.path.join(path_to_folder, folder, 'images'), pic), target_size=(200, 200), color_mode='grayscale')  # Ajustar tamaño y modo de color
                            img_array = img_to_array(img) / 255.0  # Normalizar la imagen
                            images.append(img_array)
                        
                        labels_txt = np.loadtxt(os.path.join(path_to_folder, folder, file))

                        for label in labels_txt:
                            if label == 0:
                                label = [np.array([1, 0]), np.array([np.nan]*4), np.array([np.nan])]
                            elif label == 1:
                                label = [np.array([0, 1]), np.array([np.nan]*4), np.array([np.nan])]
                            label = pad_sequences(label, dtype='float32', padding='post', value=np.nan)
                            labels.append(label)


                    elif 'fire' in file and fire < max_by_experiment:
                        fire += 1
                        for pic in sorted(os.listdir(os.path.join(path_to_folder, folder, 'images'))):
                            img = load_img(os.path.join(os.path.join(path_to_folder, folder, 'images'), pic), target_size=(200, 200), color_mode='grayscale')  # Ajustar tamaño y modo de color
                            img_array = img_to_array(img) / 255.0  # Normalizar la imagen
                            images.append(img_array)
                        
                        labels_txt = np.loadtxt(os.path.join(path_to_folder, folder, file), delimiter=' ')

                        for label in labels_txt:
                            label = [np.array([np.nan]*2), label, np.array([np.nan])]
                            label = pad_sequences(label, dtype='float32', padding='post', value=np.nan)
                            labels.append(label)

                            
                    elif 'sync' in file and steer < max_by_experiment:
                        steer += 1
                        for pic in sorted(os.listdir(os.path.join(path_to_folder, folder, 'images'))):
                            img = load_img(os.path.join(os.path.join(path_to_folder, folder, 'images'), pic), target_size=(200, 200), color_mode='grayscale')
                            img_array = img_to_array(img) / 255.0
                            images.append(img_array)

                        labels_txt = np.loadtxt(os.path.join(path_to_folder, folder, file), usecols=0, delimiter=',', skiprows=1)

                        for label in labels_txt:
                            label = [np.array([np.nan]*2), np.array([np.nan]*4), np.array([label])]
                            label = pad_sequences(label, dtype='float32', padding='post', value=np.nan)
                            labels.append(label)

    return np.array(images), np.array(labels)

images_test, labels_test = dataset_creator('../../testing', 5)
indices = np.random.permutation(images_test.shape[0])
images_test = images_test[indices]
labels_test = labels_test[indices]
y_col_test, y_fire_test, y_steer_test = labels_test[:,0, :][:, :2], labels_test[:, 1, :], labels_test[:, 2, :][:, 0]

In [8]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score

# Función para calcular precisión en tareas de clasificación
def calculate_classification_precision(predictions, labels):
    # Aplicar argmax a las predicciones y etiquetas para obtener la clase predicha
    if predictions.ndim == 2:  # Para clasificación binaria o multiclase
        predictions = np.argmax(predictions, axis=1)
    
    if labels.ndim == 2:  # Para clasificación binaria o multiclase
        labels = np.argmax(labels, axis=1)

    # Asegurarnos de que las etiquetas no tengan NaN
    valid_indices = ~np.isnan(labels)  # Indices donde el label no es NaN
    valid_labels = labels[valid_indices]
    valid_predictions = predictions[valid_indices]
    
    # Calcular la precisión
    accuracy = accuracy_score(valid_labels, valid_predictions)
    return accuracy


# Función para calcular el error absoluto medio (para la regresión)
def calculate_regression_precision(predictions, labels):
    valid_indices = ~np.isnan(labels)  # Indices donde el label no es NaN
    valid_labels = labels[valid_indices]
    valid_predictions = predictions[valid_indices]
    
    # Calcular el error absoluto medio para la regresión
    mae = np.mean(np.abs(valid_predictions - valid_labels))
    return mae

# Función para evaluar el modelo normal (no quantizado) en cada tarea
def evaluate_model_normal(model, images, labels_bin, labels_multiclass, labels_reg):
    predictions_bin = []
    predictions_multiclass = []
    predictions_reg = []

    # Iterar sobre las imágenes
    for img in images:
        pred = model(tf.expand_dims(img, axis=0))  # Expandir dimensión para batch de tamaño 1

        # Aquí procesamos la salida, que es un diccionario
        class_1_pred = pred['labels_output'].numpy()  # Clasificador binario
        class_2_pred = pred['fire_output'].numpy()  # Clasificador multiclase
        reg_pred = pred['steering_output'].numpy()  # Regresión

        # Aplanar las predicciones de cada tarea
        predictions_bin.append(class_1_pred.flatten())  # Clasificación binaria
        predictions_multiclass.append(class_2_pred.flatten())  # Clasificación multiclase
        predictions_reg.append(reg_pred.flatten())  # Regresión

    # Convertir las predicciones a numpy para calcular la precisión
    predictions_bin = np.array(predictions_bin)
    predictions_multiclass = np.array(predictions_multiclass)
    predictions_reg = np.array(predictions_reg)

    # Filtrar índices válidos para cada tarea
    valid_indices_bin = ~np.isnan(labels_bin[:, 0])  # Índices válidos para clasificación binaria
    valid_indices_multiclass = ~np.isnan(labels_multiclass[:, 0])  # Índices válidos para clasificación multiclase
    valid_indices_reg = ~np.isnan(labels_reg)  # Índices válidos para regresión

    # Aplicar el filtrado a las predicciones y etiquetas
    predictions_bin = predictions_bin[valid_indices_bin]
    labels_bin = labels_bin[valid_indices_bin]

    predictions_multiclass = predictions_multiclass[valid_indices_multiclass]
    labels_multiclass = labels_multiclass[valid_indices_multiclass]

    predictions_reg = predictions_reg[valid_indices_reg]
    labels_reg = labels_reg[valid_indices_reg]

    # Calcular la precisión para la clasificación binaria
    accuracy_bin = calculate_classification_precision(predictions_bin, labels_bin)

    # Calcular la precisión para la clasificación multiclase
    accuracy_multiclass = calculate_classification_precision(predictions_multiclass, labels_multiclass)

    # Calcular la precisión para la regresión (error absoluto medio)
    regression_precision = calculate_regression_precision(predictions_reg, labels_reg)

    return accuracy_bin, accuracy_multiclass, regression_precision

# Función para evaluar el modelo quantizado (TFLite) en cada tarea
def evaluate_model_quantized(tflite_path, images, labels_bin, labels_multiclass, labels_reg):
    # Cargar el modelo TFLite
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    # Obtener detalles de entrada y salida
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    predictions_bin = []
    predictions_multiclass = []
    predictions_reg = []

    # Iterar sobre las imágenes
    for img in images:
        # Preparar entrada
        input_data = tf.expand_dims(img, axis=0).numpy().astype(input_details[0]['dtype'])
        interpreter.set_tensor(input_details[0]['index'], input_data)

        interpreter.invoke()  # Ejecutar inferencia

        # Extraer las predicciones para cada salida
        outputs = [interpreter.get_tensor(output['index']) for output in output_details]
        
        # Aplanar las salidas para cada tarea (por ejemplo, 'labels_output', 'fire_output', 'steering_output')
        predictions_bin.append(outputs[2].flatten())  # Clasificación binaria
        predictions_multiclass.append(outputs[0].flatten())  # Clasificación multiclase
        predictions_reg.append(outputs[1].flatten())  # Regresión

    # Convertir las predicciones a numpy para calcular la precisión
    predictions_bin = np.array(predictions_bin)
    predictions_multiclass = np.array(predictions_multiclass)
    predictions_reg = np.array(predictions_reg)

    # Calcular la precisión para la clasificación binaria
    valid_indices_bin = ~np.isnan(labels_bin[:, 0])
    valid_indices_multiclass = ~np.isnan(labels_multiclass[:, 0])
    valid_indices_reg = ~np.isnan(labels_reg)

    accuracy_bin = calculate_classification_precision(predictions_bin[valid_indices_bin], labels_bin[valid_indices_bin])
    accuracy_multiclass = calculate_classification_precision(predictions_multiclass[valid_indices_multiclass], labels_multiclass[valid_indices_multiclass])
    regression_precision = calculate_regression_precision(predictions_reg[valid_indices_reg], labels_reg[valid_indices_reg])

    return accuracy_bin, accuracy_multiclass, regression_precision


# Cargar el modelo original (sin quantizar)
model = TFSMLayer('original_model', call_endpoint='serving_default')  # Asegúrate de que el modelo se cargue correctamente

# Evaluar el modelo original
accuracy_bin, accuracy_multiclass, regression_precision = evaluate_model_normal(
    model, images_test, y_col_test, y_fire_test, y_steer_test
)

# Evaluar el modelo quantizado
tflite_path = 'qat_model.tflite'  # Ruta a tu modelo quantizado en formato TFLite
accuracy_bin_qat, accuracy_multiclass_qat, regression_precision_qat = evaluate_model_quantized(
    tflite_path, images_test, y_col_test, y_fire_test, y_steer_test
)

# Imprimir resultados
print(f"Precisión clasificador binario (original): {accuracy_bin}")
print(f"Precisión clasificador multiclase (original): {accuracy_multiclass}")
print(f"Precisión regresión (original): {regression_precision}")

print(f"Precisión clasificador binario (quantizado): {accuracy_bin_qat}")
print(f"Precisión clasificador multiclase (quantizado): {accuracy_multiclass_qat}")
print(f"Precisión regresión (quantizado): {regression_precision_qat}")


2024-12-05 16:36:37.005713: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Precisión clasificador binario (original): 0.7709637046307884
Precisión clasificador multiclase (original): 0.7940436241610739
Precisión regresión (original): 0.18170204758644104
Precisión clasificador binario (quantizado): 0.9136420525657072
Precisión clasificador multiclase (quantizado): 0.738255033557047
Precisión regresión (quantizado): 0.2646496891975403


In [9]:
pd.DataFrame({'Metric': ['Collision accuracy', 'Fire detection accuracy', 'Steering angle MSE', 'Number of epochs'],
               'Original model':[accuracy_bin, accuracy_multiclass, regression_precision, 21], 
              'QAT model': [accuracy_bin_qat, accuracy_multiclass_qat, regression_precision_qat, 45]}).to_csv('performance_table.csv', index=False)